In [1]:
import os
import torch

import scanpy as sc
import numpy as np
import pandas as pd
import gseapy as gp

from tqdm import tqdm
from sklearn.cluster import KMeans
from contrastive_vi.model.contrastive_vi import ContrastiveVIModel
from scripts import constants
from scvi._settings import settings

Global seed set to 0
During startup - Warning messages:
1: package ‘methods’ was built under R version 3.6.1 
2: package ‘datasets’ was built under R version 3.6.1 
3: package ‘utils’ was built under R version 3.6.1 
4: package ‘grDevices’ was built under R version 3.6.1 
5: package ‘graphics’ was built under R version 3.6.1 
6: package ‘stats’ was built under R version 3.6.1 


In [2]:
settings.seed = 0
device = "cuda:1"
dataset = "mcfarland_2020"

Global seed set to 0


In [3]:
pathway_enr_fdr = 0.05
expression_delta = 0.15

In [4]:
split_key = constants.DATASET_SPLIT_LOOKUP[dataset]["split_key"]
background_value = constants.DATASET_SPLIT_LOOKUP[dataset]["background_value"]
label_key = constants.DATASET_SPLIT_LOOKUP[dataset]["label_key"]
seeds = constants.DEFAULT_SEEDS
latent_size = 10

In [5]:
adata = sc.read_h5ad(
    os.path.join(
        constants.DEFAULT_DATA_PATH,
        f"{dataset}/preprocessed/adata_top_2000_genes_tc.h5ad",
    )
)
ContrastiveVIModel.setup_anndata(adata, layer="count")

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


INFO     No batch_key inputted, assuming all cells are same batch                            
INFO     No label_key inputted, assuming all cells have same label                           
INFO     Using data from adata.layers["count"]                                               
INFO     Successfully registered anndata object containing 5928 cells, 2000 vars, 1 batches, 
         1 labels, and 0 proteins. Also registered 0 extra categorical covariates and 0 extra
         continuous covariates.                                                              
INFO     Please do not further modify adata until model is trained.                          


In [6]:
target_indices = np.where(adata.obs[split_key] != background_value)[0]
background_indices = np.where(adata.obs[split_key] == background_value)[0]

In [7]:
genes = pd.read_table(
    os.path.join(
        constants.DEFAULT_DATA_PATH,
        dataset,
        "idasanutlin",
        "Idasanutlin_24hr_expt1",
        "genes.tsv",
    ),
    header=None,
)
genes = genes.rename(columns={0: "ensembl_id", 1: "gene_symbol"})
genes = genes[genes["ensembl_id"].isin(adata.var.index)]

In [8]:
model_list = []
for seed in tqdm(seeds):
    result_dir = os.path.join(
        constants.DEFAULT_RESULTS_PATH,
        f"{dataset}/contrastiveVI/latent_{latent_size}",
        f"{seed}",
    )
    model_list.append(
        torch.load(
            os.path.join(result_dir, "model.ckpt"),
            map_location=device,
        ),
    )

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:22<00:00,  4.47s/it]


In [9]:
de_result_list = []
enr_result_list = []

for seed_index, seed in enumerate(seeds):
    model = model_list[seed_index]
    
    de_result = model.differential_expression(
        adata=adata,
        groupby=None,
        group1=None,
        group2=None,
        idx1=background_indices,
        idx2=target_indices,
        mode="change",
        delta=expression_delta,
        batch_size=128,
        all_stats=True,
        batch_correction=False,
        batchid1=None,
        batchid2=None,
        fdr_target=0.05,
        silent=False,
        target_idx=target_indices,
    )

    de_result.reset_index()
    de_result["ensembl_id"] = de_result.index
    de_result = de_result.merge(genes, on="ensembl_id")
    de_result["seed"] = seed
    de_result_list.append(de_result)

    top_genes = de_result[de_result["proba_de"] > 0.95]["gene_symbol"].tolist()
    enr = gp.enrichr(
        gene_list=top_genes,
        gene_sets="KEGG_2016",
        organism="human",
        cutoff=pathway_enr_fdr,
    )
    enr_result = enr.results
    enr_result = enr_result[enr_result["Adjusted P-value"] < pathway_enr_fdr]
    enr_result["seed"] = seed
    enr_result_list.append(enr_result)

DE...: 100%|██████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:33<00:00, 33.70s/it]


/tmp/ipykernel_20130/2342364654.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enr_result["seed"] = seed


DE...: 100%|██████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:34<00:00, 34.13s/it]


/tmp/ipykernel_20130/2342364654.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enr_result["seed"] = seed


DE...: 100%|██████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:33<00:00, 33.66s/it]


/tmp/ipykernel_20130/2342364654.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enr_result["seed"] = seed


DE...: 100%|██████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:33<00:00, 33.53s/it]


/tmp/ipykernel_20130/2342364654.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enr_result["seed"] = seed


DE...: 100%|██████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:33<00:00, 33.59s/it]


/tmp/ipykernel_20130/2342364654.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enr_result["seed"] = seed


In [10]:
enr_df = pd.concat(enr_result_list)
cols = ["Gene_set", "Term", "Adjusted P-value", "Overlap", "Genes"]
cols += ["seed"]
enr_df = enr_df[cols]

In [11]:
enr_df

,Gene_set,Term,Adjusted P-value,Overlap,Genes,seed
0,KEGG_2016,Rheumatoid arthritis Homo sapiens hsa05323,0.000154,18/90,CXCL6;IL11;HLA-DRB5;MMP1;TGFB3;CCL20;MMP3;CXCL...,123
1,KEGG_2016,Amoebiasis Homo sapiens hsa05146,0.001564,17/100,SERPINB3;SERPINB4;SERPINB1;LAMB3;TGFB3;IL1R2;L...,123
2,KEGG_2016,Protein digestion and absorption Homo sapiens ...,0.004189,15/90,CELA3A;PRSS1;COL15A1;COL27A1;COL1A1;COL3A1;SLC...,123
3,KEGG_2016,ECM-receptor interaction Homo sapiens hsa04512,0.004207,14/82,LAMB3;ITGA2;LAMA4;LAMA3;FN1;LAMC2;THBS2;COL1A1...,123
4,KEGG_2016,Cytokine-cytokine receptor interaction Homo sa...,0.005461,29/265,CXCL6;CSF3;IL24;CXCL1;CXCL13;CXCL3;CXCL14;CXCL...,123
5,KEGG_2016,Complement and coagulation cascades Homo sapie...,0.027257,12/79,C3;SERPINB2;CFH;C1S;PLAU;SERPINE1;FGG;BDKRB1;P...,123
6,KEGG_2016,Graft-versus-host disease Homo sapiens hsa05332,0.033881,8/41,HLA-DRB5;IL6;IL1B;PRF1;GZMB;KLRC1;HLA-G;HLA-DRB1,123
7,KEGG_2016,Pertussis Homo sapiens hsa05133,0.045107,11/75,PYCARD;C3;CXCL6;IL6;CALML5;C1S;IL1B;CALML3;C4B...,123
0,KEGG_2016,Rheumatoid arthritis Homo sapiens hsa05323,0.002108,15/90,CXCL6;IL11;MMP1;TGFB3;CCL20;MMP3;CXCL1;IL6;CXC...,42
1,KEGG_2016,Protein digestion and absorption Homo sapiens ...,0.002108,15/90,COL17A1;PRSS1;MME;COL11A1;COL1A1;COL3A1;SLC7A7...,42


## Aggregate analysis

In [12]:
de_result = pd.concat(de_result_list)
de_result_mean = (
    de_result.groupby("gene_symbol", as_index=False)
    .mean()
    .sort_values(by="proba_de", ascending=False)
)

In [13]:
de_result_mean["proba_de"].describe()

count    2000.000000
mean        0.945427
std         0.015388
min         0.847520
25%         0.940150
50%         0.948800
75%         0.954960
max         0.971680
Name: proba_de, dtype: float64

In [14]:
top_genes = de_result_mean[de_result_mean["proba_de"] > 0.95]["gene_symbol"].tolist()

enr = gp.enrichr(
    gene_list=top_genes,
    gene_sets="KEGG_2016",
    organism="human",
    cutoff=0.05,
)
enr_results = enr.results
enr_results = enr_results[enr_results["Adjusted P-value"] < 0.05]

In [15]:
enr_results

,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes
0,KEGG_2016,Rheumatoid arthritis Homo sapiens hsa05323,18/90,7.638178e-08,0.000016,0,0,5.438571,89.124707,CXCL6;IL11;HLA-DRB5;MMP1;TGFB3;CCL20;MMP3;CXCL...
1,KEGG_2016,Cytokine-cytokine receptor interaction Homo sa...,30/265,2.936513e-06,0.000317,0,0,2.791647,35.560805,CXCL6;IL26;IL24;TNFRSF11B;CXCL1;CXCL13;CXCL3;C...
2,KEGG_2016,Amoebiasis Homo sapiens hsa05146,16/100,8.694852e-06,0.000626,0,0,4.131618,48.144828,SERPINB3;SERPINB4;SERPINB1;LAMB3;TGFB3;IL1R2;L...
3,KEGG_2016,ECM-receptor interaction Homo sapiens hsa04512,14/82,1.466807e-05,0.000792,0,0,4.459379,49.632162,LAMB3;ITGA2;LAMA4;TNC;FN1;LAMC2;HMMR;THBS2;COL...
4,KEGG_2016,Protein digestion and absorption Homo sapiens ...,14/90,4.331468e-05,0.001871,0,0,3.988294,40.070467,COL17A1;PRSS1;COL27A1;COL1A1;COL3A1;SLC7A7;COL...
5,KEGG_2016,Graft-versus-host disease Homo sapiens hsa05332,8/41,3.936748e-04,0.014172,0,0,5.224859,40.962817,HLA-DRB5;IL6;IL1B;PRF1;GZMB;KLRC1;HLA-G;HLA-DRB1
6,KEGG_2016,Pertussis Homo sapiens hsa05133,11/75,4.736015e-04,0.014614,0,0,3.710902,28.407491,PYCARD;CXCL6;IL6;CALML5;SFTPA2;C1S;IL1B;CALML3...
7,KEGG_2016,TNF signaling pathway Homo sapiens hsa04668,13/110,1.243597e-03,0.033577,0,0,2.895150,19.367821,EDN1;CCL20;MMP3;CXCL1;CXCL3;PTGS2;MMP9;CXCL5;I...
8,KEGG_2016,Salivary secretion Homo sapiens hsa04970,11/89,2.007275e-03,0.048175,0,0,3.042604,18.897546,GUCY1A3;BST1;CST1;CALML5;DMBT1;KCNMA1;ITPR1;CA...


In [16]:
len(top_genes)

893